# Spoof Attack Analysis


In [1]:
import re
from typing import Dict, Any

import wandb
import polars as pl
import altair as alt

In [2]:
WANDB_ENTITY = "juanbelieni-lab"
WANDB_PROJECT = "watermark-spoof"
SWEEP_ID = "excgqv1x"

In [3]:
def parse_sweep_url(url: str):
    m = re.search(r'wandb\.ai/([^/]+)/([^/]+)/sweeps/([a-z0-9]+)', url)
    if not m:
        raise ValueError('Unrecognized sweep URL; expected .../ENTITY/PROJECT/sweeps/ID')
    return m.group(1), m.group(2), m.group(3)

def fetch_sweep_runs(entity: str, project: str, sweep_id: str) -> pl.DataFrame:
    """
    Returns a DataFrame where each row is a run.
    Columns include run metadata plus flattened `config.*` and `summary.*` fields.
    """

    api = wandb.Api(timeout=300)
    sw = api.sweep(f'{entity}/{project}/{sweep_id}')
    runs = list(sw.runs)
    records = []
    for r in runs:
        cfg: Dict[str, Any] = dict(getattr(r, 'config', {}) or {})
        summ: Dict[str, Any] = dict(getattr(r, 'summary', {}) or {})
        rec: Dict[str, Any] = {
            'run_id': getattr(r, 'id', None),
            'run_name': getattr(r, 'name', None),
            'state': getattr(r, 'state', None),
            'created_at': str(getattr(r, 'created_at', '')),
            'entity': entity,
            'project': project,
            'sweep_id': sweep_id,
        }

        items = [*cfg.items(), *summ.items()]

        for k, v in items:
            rec[k] = v

        records.append(rec)
    df = pl.DataFrame(records)
    return df


In [4]:
# Fetch runs into a DataFrame
runs_df = fetch_sweep_runs(WANDB_ENTITY, WANDB_PROJECT, SWEEP_ID)
print(f'Loaded {len(runs_df)} runs from sweep.')
runs_df.head()


wandb: Currently logged in as: juanbelieni (juanbelieni-lab) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Loaded 160 runs from sweep.


run_id,run_name,state,created_at,entity,project,sweep_id,seed,clip_c,model_id,delta_att,num_prompts,window_size,limit_samples,max_new_tokens,_runtime,_step,_timestamp,_wandb,success_rate,z_avg
str,str,str,str,str,str,str,i64,i64,str,f64,i64,i64,i64,i64,i64,i64,f64,object,f64,f64
"""9wv71qju""","""distinctive-sweep-1""","""finished""","""2025-08-27T21:05:19Z""","""juanbelieni-lab""","""watermark-spoof""","""excgqv1x""",0,2,"""meta-llama/Llama-3.2-3B-Instru…",2.5,100,2,100,500,72,0,1.7563e9,{'runtime': 72},0.14,0.653634
"""13gagesh""","""electric-sweep-2""","""finished""","""2025-08-27T21:07:24Z""","""juanbelieni-lab""","""watermark-spoof""","""excgqv1x""",1,2,"""meta-llama/Llama-3.2-3B-Instru…",2.5,100,2,100,500,72,0,1.7563e9,{'runtime': 72},0.13,0.475277
"""baweqmac""","""honest-sweep-3""","""finished""","""2025-08-27T21:08:46Z""","""juanbelieni-lab""","""watermark-spoof""","""excgqv1x""",2,2,"""meta-llama/Llama-3.2-3B-Instru…",2.5,100,2,100,500,71,0,1.7563e9,{'runtime': 71},0.14,0.707484
"""wqffuvhj""","""iconic-sweep-4""","""finished""","""2025-08-27T21:10:09Z""","""juanbelieni-lab""","""watermark-spoof""","""excgqv1x""",3,2,"""meta-llama/Llama-3.2-3B-Instru…",2.5,100,2,100,500,72,0,1.7563e9,{'runtime': 72},0.12,0.455922
"""3rlwy1ce""","""breezy-sweep-5""","""finished""","""2025-08-27T21:11:32Z""","""juanbelieni-lab""","""watermark-spoof""","""excgqv1x""",4,2,"""meta-llama/Llama-3.2-3B-Instru…",2.5,100,2,100,500,72,0,1.7563e9,{'runtime': 72},0.15,0.601399


In [12]:
# Group by limit_samples and delta_att using Polars, then plot lines colored by delta_att
agg_df = (
    runs_df
    .filter(pl.col("limit_samples") <= 1000)
    .group_by(["limit_samples", "delta_att"])
    .agg(
      pl.col("success_rate").mean().alias("success_rate"),
      pl.col("z_avg").mean().alias("z_avg"),
    )
    .sort(["delta_att", "limit_samples"])
)

alt.Chart(agg_df.to_pandas()).mark_line().encode(
    x=alt.X("limit_samples", type="quantitative"),
    y=alt.Y("success_rate", type="quantitative"),
    color=alt.Color("delta_att", type="nominal", title="delta_att"),
)

alt.Chart(...)